In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse

In [14]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 200)

In [2]:
url = 'https://bilim.akipress.org/profiles:3/school/list'

In [ ]:
#https://bilim.akipress.org/profiles:3/school/list//page:2
#https://bilim.akipress.org/profiles:3/school/list//page:23

In [ ]:
#url = https://www.drugbank.ca/drugs?page=" + str(page_number)

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text, "html.parser")

In [5]:
# find where the data we want resides (a tag, class name, table number)
table = soup.select('table', class_='table-striped')

In [6]:
table = table[0]

In [7]:
# find_all tr
schools_trs = soup.select('tr')

In [8]:
# separate the first tr tag row for the header
#schools_th = schools_trs[0].select('th')
#schools_header = []
#for (i,th) in enumerate(schools_th):
#    if(i >= 1):
#        schools_header.append(th.text)
#schools_header

['Наименование', 'Адрес']

In [10]:
# for each tr, find tds then for each td get text inside, then save to new array
schools_list = []
for tr in schools_trs[1:]:
    tds = tr.select('td')
    td_list = []
    for (i, td) in enumerate(tds):
        if(i >= 1):
            td_list.append(td.text)
        #if it's the second column, get the href link also
        if(i == 1):
            td_list.append(td.find('a')['href']) 
    schools_list.append(td_list)

In [11]:
df = pd.DataFrame(schools_list, columns=["name", "link", "address"])

In [15]:
df['urls'] = ("https://bilim.akipress.org"+df['link'])

In [16]:
df['contents'] = df.urls.apply(lambda url: requests.get(url).text)

In [34]:
def get_score(content):
    soup = BeautifulSoup(content)
    tables = soup.select("table")
    try:
        trs = tables[2].find_all('tr')
        for (i, tr) in enumerate (trs):
           #if it's the second row, go there
            if (i == 1):
                tds = tr.find_all('td')
                for (i, td) in enumerate(tds):
           # if it's the third column, get the value
                    if(i == 2):
                        score = td.text
        return score
    except:
        pass

In [35]:
score = df.contents.apply(get_score)
score = score.to_frame().reset_index()
score.columns = ['index', 'value']

In [36]:
df['score'] = score['value']
df

,name,link,address,urls,contents,score
0,СШ им.Огонбаева,/profile:715/ssh-imogonbaeva/?from=bilim&place=list,"Таласская область, Таласский район c.Кек-Кашат 724009 ул. Школьная 1",https://bilim.akipress.org/profile:715/ssh-imogonbaeva/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>СШ им.Огонбаева - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta name=""...",96.8
1,УВК школа-гимназия №70,/profile:2186/uvk-shkolagimnaziya-70/?from=bilim&place=list,"Бишкек, Первомайский район Боконбаева, 153",https://bilim.akipress.org/profile:2186/uvk-shkolagimnaziya-70/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>УВК школа-гимназия №70 - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta...",нет
2,УВК школа-гимназия №17 им. А.Пушкина,/profile:10/uvk-shkolagimnaziya-17-im-apushkina/?from=bilim&place=list,"Бишкек, Октябрьский район Бишкек, 4 мкр, 31",https://bilim.akipress.org/profile:10/uvk-shkolagimnaziya-17-im-apushkina/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>УВК школа-гимназия №17 им. А.Пушкина - адрес, новости, отзывы, выпускники — Билим АКИpress</tit...",109.3
3,Учебный комплекс авторской физико-математической школы-лицея №61 Е.Якира,/profile:2188/uchebnyi-kompleks-avtorskoi-fizikomatematicheskoi/?from=bilim&place=list,"Бишкек, Первомайский район Малдыбаева, 10",https://bilim.akipress.org/profile:2188/uchebnyi-kompleks-avtorskoi-fizikomatematicheskoi/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>Учебный комплекс авторской физико-математической школы-лицея №61 Е.Якира - адрес, новости, отзы...",нет
4,Школа-гимназия №91,/profile:2172/shkolagimnaziya-91/?from=bilim&place=list,"Бишкек, Октябрьский район ул.Оберона б/н",https://bilim.akipress.org/profile:2172/shkolagimnaziya-91/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>Школа-гимназия №91 - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta nam...",156
...,...,...,...,...,...,...
95,СШ им.Кудайбердиева,/profile:1082/ssh-imkudaiberdieva/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Кашат Советская б\н 722320",https://bilim.akipress.org/profile:1082/ssh-imkudaiberdieva/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>СШ им.Кудайбердиева - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta na...",114.9
96,СШ им.Керимгазиева,/profile:1083/ssh-imkerimgazieva/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Темировка Иманалива б\н 722319",https://bilim.akipress.org/profile:1083/ssh-imkerimgazieva/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>СШ им.Керимгазиева - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta nam...",109.8
97,СШ им. Колесникова,/profile:1080/ssh-im-kolesnikova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Семеновка ул.Алмакучукова б\н 722323",https://bilim.akipress.org/profile:1080/ssh-im-kolesnikova/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>СШ им. Колесникова - адрес, новости, отзывы, выпускники — Билим АКИpress</title>\n <meta nam...",100.8
98,СШ им.Маматова,/profile:1075/ssh-immamatova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Жаркынбаево ул.Абдракманова б\н 722326",https://bilim.akipress.org/profile:1075/ssh-immamatova/?from=bilim&place=list,"\r\n<!doctype html>\n<html lang=""ru"" data-tip=""222"">\n<head>\n <meta charset=""UTF-8"">\n <title>СШ им.Маматова - адре

In [39]:
df1=df

In [40]:
del df1['contents']

In [42]:
df1.to_csv('scores.csv', index = False)